In [1]:
import pandas as pd
from pickle import load
from pickle import dump
import numpy as np
pd.set_option("max_rows", None)
pd.set_option("max_columns", None)

In [2]:
# Templates for finding stuff
# df_race.where(df['County Name'] is in df_race['County'].values).notna()
# df['County Name'].isin(df_race['County']).value_counts()
# (df.loc[df['County Name'].str.contains('Oglala', case=False)])
# df_race.loc[~df_race['County'].isin(df['County Name'])]

Load in race file from other notebook.

In [3]:
df_race = load(open('df_race.pkl', 'rb'))

Load in election results from data.world.

In [4]:
df = pd.read_csv('2016 Presidential Election Analysis.csv')

Remove punctuation from column names.  
Remove commas from values.  
Drop irrelevant or redundant columns.  
Replace state names to match census data.  
Remove the word county from County values.  
Adjust Oglala to Oglala Lakota to match census data.  
Remove space in La Salle to match census data.  

In [5]:
df.columns = df.columns.str.strip().str.replace('[^\w\s]', '')

<ipython-input-5-29d73bdc17d5>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.strip().str.replace('[^\w\s]', '')


In [6]:
def remove_punctuation(x):
    try:
        x = x.str.replace('[^\w\s]','')
    except:
        pass
    return x

In [7]:
df = df.apply(remove_punctuation)

<ipython-input-6-fabae1a16f96>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  x = x.str.replace('[^\w\s]','')


In [8]:
df = df.replace(',', '', regex = True) #can this be removed?

In [9]:
irrelevant_columns = df.loc[:, 'CBSA Code': 'Metropolitan Division Code']

In [10]:
df.drop('County Population', axis = 1, inplace = True)

In [11]:
df.drop(irrelevant_columns, axis = 1, inplace = True)

In [12]:
us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
}

In [13]:
df['State Code'] = df['State Code'].map(us_state_abbrev)

In [14]:
df['County Name'] = df['County Name'].str.rstrip('County')

In [15]:
df = df.replace('Oglala', 'Oglala Lakota', regex = True)

In [16]:
df = df.replace('La Salle Parish', 'LaSalle Parish', regex = True)

Check to make sure County columns match in results and race.

In [17]:
missing = df.loc[~df['County Name'].isin(df_race['County'])]
missing

,State Code,County Name,Clinton or Trump State,Clinton,Trump,Total,Clinton,Trump,Vote Difference CT,Vote Difference TC,Percent Difference CT,Percent Difference TC,Obama,Romney,2012 Total Votes,Obama,Romney,Vote Difference OR,Difference OR,Clinton BW Obama,Trump BW Romney,Clinton BW Obama,Trump BW Romney,2010 Land Area,Density,Combined FIPS Code,ANSI Code,State FIPS Code,County FIPS Code,CentralOutlying County
67,Alaska,Alaska,Trump,93007,130415,246588,3772,5289,37408,37408,1517,1517,122640,164676,300495,4081,5480,42036,1399,29633,34261,310,191,570641,1,Alaska,Alaska,NaN,NaN,ZAlaska


In [18]:
df_race.loc[~df_race['County'].isin(df['County Name'])]

1,id,total_pop,total_pop_one_race,pop_white,pop_african_american,pop_native,pop_asian,pop_islander,pop_other,total_pop_two_races,County,State


Rename state and county columns to match census data.

In [19]:
df.rename(columns={"State Code": "State", "County Name": "County"}, inplace = True)

In [20]:
df['State'] = df['State'].fillna(df['County'])

Dropping Alaska for now. Will need to come back and figure it out. Maybe just create an entire dataframe for Alaska and then add it back in? Rather than manually adding the row, do a sum row without object columns and add those columns back in.

In [21]:
alaska1 = df.loc[df['State'].str.contains('Alaska', case=False)]

In [22]:
df.drop(df.loc[df['State'].str.contains('Alaska', case=False)].index, inplace = True)

In [23]:
df.head()

,State,County,Clinton or Trump State,Clinton,Trump,Total,Clinton,Trump,Vote Difference CT,Vote Difference TC,Percent Difference CT,Percent Difference TC,Obama,Romney,2012 Total Votes,Obama,Romney,Vote Difference OR,Difference OR,Clinton BW Obama,Trump BW Romney,Clinton BW Obama,Trump BW Romney,2010 Land Area,Density,Combined FIPS Code,ANSI Code,State FIPS Code,County FIPS Code,CentralOutlying County
0,Alabama,Autauga,Trump,5908,18110,24661,2396,7344,12202,12202,4948,4948,6354,17366,23909,2658,7263,11012,4606,446,744,262,080,594,93,1001,01001,1.0,1.0,Central
1,Alabama,Baldwin,Trump,18409,72780,94090,1957,7735,54371,54371,5779,5779,18329,65772,84988,2157,7739,47443,5582,80,7008,200,004,1590,128,1003,01003,1.0,3.0,Central
2,Alabama,Barbour,Trump,4848,5431,10390,4666,5227,583,583,561,561,5873,5539,11459,5125,4834,334,291,1025,108,459,393,885,30,1005,01005,1.0,5.0,Central
3,Alabama,Bibb,Trump,1874,6733,8748,2142,7697,4859,4859,5554,5554,2200,6131,8391,2622,7307,3931,4685,326,602,480,390,623,36,1007,01007,1.0,7.0,Outlying
4,Alabama,Blount,Trump,2150,22808,25384,847,8985,20658,20658,8138,8138,2961,20741,23980,1235,8649,17780,7415,811,2067,388,336,645,89,1009,01009,1.0,9.0,Outlying


This target was moved to a different notebook.

In [24]:
target_to_drop = df.iloc[:, [2, 3, 4, 6, 7, 8, 9, 10, 11]]
target_to_drop.head()

,Clinton or Trump State,Clinton,Trump,Clinton,Trump,Vote Difference CT,Vote Difference TC,Percent Difference CT,Percent Difference TC
0,Trump,5908,18110,2396,7344,12202,12202,4948,4948
1,Trump,18409,72780,1957,7735,54371,54371,5779,5779
2,Trump,4848,5431,4666,5227,583,583,561,561
3,Trump,1874,6733,2142,7697,4859,4859,5554,5554
4,Trump,2150,22808,847,8985,20658,20658,8138,8138


In [25]:
state = pd.DataFrame(df.State)
county = pd.DataFrame(df.County)

In [26]:
target_to_move = pd.concat([target_to_drop, state, county], axis = 1)

In [27]:
target_to_move.head()

,Clinton or Trump State,Clinton,Trump,Clinton,Trump,Vote Difference CT,Vote Difference TC,Percent Difference CT,Percent Difference TC,State,County
0,Trump,5908,18110,2396,7344,12202,12202,4948,4948,Alabama,Autauga
1,Trump,18409,72780,1957,7735,54371,54371,5779,5779,Alabama,Baldwin
2,Trump,4848,5431,4666,5227,583,583,561,561,Alabama,Barbour
3,Trump,1874,6733,2142,7697,4859,4859,5554,5554,Alabama,Bibb
4,Trump,2150,22808,847,8985,20658,20658,8138,8138,Alabama,Blount


In [28]:
df.drop(target_to_drop, axis = 1, inplace = True)

Remove some redundant columns now that I understand them. These are all just different calculations based on the vote totals. Surely that wouldn't actually be helpful... would it? Ask Max, I guess.

In [29]:
drop = df.iloc[:, 6:14]
df.drop(drop, axis = 1, inplace = True)

In [30]:
df.head()

,State,County,Total,Obama,Romney,2012 Total Votes,2010 Land Area,Density,Combined FIPS Code,ANSI Code,State FIPS Code,County FIPS Code,CentralOutlying County
0,Alabama,Autauga,24661,6354,17366,23909,594,93,1001,01001,1.0,1.0,Central
1,Alabama,Baldwin,94090,18329,65772,84988,1590,128,1003,01003,1.0,3.0,Central
2,Alabama,Barbour,10390,5873,5539,11459,885,30,1005,01005,1.0,5.0,Central
3,Alabama,Bibb,8748,2200,6131,8391,623,36,1007,01007,1.0,7.0,Outlying
4,Alabama,Blount,25384,2961,20741,23980,645,89,1009,01009,1.0,9.0,Outlying


Double check everything. Should be 3112 rows without Alaska. Should not include Trump/Clinton columns.

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3112 entries, 0 to 3112
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   3112 non-null   object 
 1   County                  3112 non-null   object 
 2   Total                   3112 non-null   object 
 3   Obama                   3112 non-null   object 
 4   Romney                  3112 non-null   object 
 5   2012 Total Votes        3112 non-null   object 
 6   2010 Land Area          3112 non-null   int64  
 7   Density                 3112 non-null   int64  
 8   Combined FIPS Code      3112 non-null   object 
 9   ANSI Code               3112 non-null   object 
 10  State FIPS Code         3112 non-null   float64
 11  County FIPS Code        3112 non-null   float64
 12  CentralOutlying County  3112 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 340.4+ KB


In [32]:
df.head()

,State,County,Total,Obama,Romney,2012 Total Votes,2010 Land Area,Density,Combined FIPS Code,ANSI Code,State FIPS Code,County FIPS Code,CentralOutlying County
0,Alabama,Autauga,24661,6354,17366,23909,594,93,1001,01001,1.0,1.0,Central
1,Alabama,Baldwin,94090,18329,65772,84988,1590,128,1003,01003,1.0,3.0,Central
2,Alabama,Barbour,10390,5873,5539,11459,885,30,1005,01005,1.0,5.0,Central
3,Alabama,Bibb,8748,2200,6131,8391,623,36,1007,01007,1.0,7.0,Outlying
4,Alabama,Blount,25384,2961,20741,23980,645,89,1009,01009,1.0,9.0,Outlying


Rename some columns for ease of coding.

In [33]:
df.rename(columns = {'Total': '2016_total_votes', '2012 Total Votes':'2012_total_votes', 'Combined FIPS Code': 'combined_fips_code', 'ANSI Code': 'ansi_code', 'State FIPS Code': 'state_fips_code', 'County FIPS Code': 'county_fips_code', 'CentralOutlying County': 'central_outlying', '2010 Land Area': '2010_land_area'}, inplace = True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3112 entries, 0 to 3112
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               3112 non-null   object 
 1   County              3112 non-null   object 
 2   2016_total_votes    3112 non-null   object 
 3   Obama               3112 non-null   object 
 4   Romney              3112 non-null   object 
 5   2012_total_votes    3112 non-null   object 
 6   2010_land_area      3112 non-null   int64  
 7   Density             3112 non-null   int64  
 8   combined_fips_code  3112 non-null   object 
 9   ansi_code           3112 non-null   object 
 10  state_fips_code     3112 non-null   float64
 11  county_fips_code    3112 non-null   float64
 12  central_outlying    3112 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 340.4+ KB


In [35]:
df.head()

,State,County,2016_total_votes,Obama,Romney,2012_total_votes,2010_land_area,Density,combined_fips_code,ansi_code,state_fips_code,county_fips_code,central_outlying
0,Alabama,Autauga,24661,6354,17366,23909,594,93,1001,01001,1.0,1.0,Central
1,Alabama,Baldwin,94090,18329,65772,84988,1590,128,1003,01003,1.0,3.0,Central
2,Alabama,Barbour,10390,5873,5539,11459,885,30,1005,01005,1.0,5.0,Central
3,Alabama,Bibb,8748,2200,6131,8391,623,36,1007,01007,1.0,7.0,Outlying
4,Alabama,Blount,25384,2961,20741,23980,645,89,1009,01009,1.0,9.0,Outlying


In [36]:
df.iloc[:, 2:-3] = df.iloc[:, 2:-3].apply(pd.to_numeric)

In [37]:
df.iloc[:, 2:-3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3112 entries, 0 to 3112
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   2016_total_votes    3112 non-null   int64
 1   Obama               3112 non-null   int64
 2   Romney              3112 non-null   int64
 3   2012_total_votes    3112 non-null   int64
 4   2010_land_area      3112 non-null   int64
 5   Density             3112 non-null   int64
 6   combined_fips_code  3112 non-null   int64
 7   ansi_code           3112 non-null   int64
dtypes: int64(8)
memory usage: 218.8 KB


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3112 entries, 0 to 3112
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               3112 non-null   object 
 1   County              3112 non-null   object 
 2   2016_total_votes    3112 non-null   int64  
 3   Obama               3112 non-null   int64  
 4   Romney              3112 non-null   int64  
 5   2012_total_votes    3112 non-null   int64  
 6   2010_land_area      3112 non-null   int64  
 7   Density             3112 non-null   int64  
 8   combined_fips_code  3112 non-null   int64  
 9   ansi_code           3112 non-null   int64  
 10  state_fips_code     3112 non-null   float64
 11  county_fips_code    3112 non-null   float64
 12  central_outlying    3112 non-null   object 
dtypes: float64(2), int64(8), object(3)
memory usage: 340.4+ KB


In [39]:
df.iloc[[-1]]

,State,County,2016_total_votes,Obama,Romney,2012_total_votes,2010_land_area,Density,combined_fips_code,ansi_code,state_fips_code,county_fips_code,central_outlying
3112,Wyoming,Weston,3334,422,2821,3323,2398,3,56045,56045,56.0,45.0,Not Metro or Micro Presumed Rural


In [40]:
df.iloc[:, 8:-1] = df.iloc[:, 8:-1].astype(str)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3112 entries, 0 to 3112
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   State               3112 non-null   object
 1   County              3112 non-null   object
 2   2016_total_votes    3112 non-null   int64 
 3   Obama               3112 non-null   int64 
 4   Romney              3112 non-null   int64 
 5   2012_total_votes    3112 non-null   int64 
 6   2010_land_area      3112 non-null   int64 
 7   Density             3112 non-null   int64 
 8   combined_fips_code  3112 non-null   object
 9   ansi_code           3112 non-null   object
 10  state_fips_code     3112 non-null   object
 11  county_fips_code    3112 non-null   object
 12  central_outlying    3112 non-null   object
dtypes: int64(6), object(7)
memory usage: 340.4+ KB


In [42]:
df.shape

(3112, 13)

Exporting files

In [43]:
dump(target_to_move, open('target_df.pkl', 'wb'))

In [44]:
dump(df, open('df_results.pkl', 'wb'))